In [2]:
%mkdir /content/drive/MyDrive/PlantFactoryTutorial

In [4]:
%cd /content/drive/MyDrive/PlantFactoryTutorial

In [6]:
%pip install icrawler

  Created wheel for icrawler: filename=icrawler-0.6.4-py2.py3-none-any.whl size=35063 sha256=bebe6123cfe2e5c74b3744ad53202579779d2266c83b3a0118a406e3eafa6b92
  Stored in directory: /root/.cache/pip/wheels/57/f7/08/e89027083863374b83f7b3e3ce52a9afa1674424a379c7faf9
Successfully built icrawler


In [7]:
from icrawler.builtin import BingImageCrawler
# 猫の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "cat"})
crawler.crawl(keyword="猫", max_num=100)
# 犬の画像を100枚取得
crawler = BingImageCrawler(storage={"root_dir": "dog"})
crawler.crawl(keyword="犬", max_num=100)

2021-06-21 00:27:06,210 - INFO - icrawler.crawler - start crawling...
2021-06-21 00:27:06,211 - INFO - icrawler.crawler - starting 1 feeder threads...
2021-06-21 00:27:06,216 - INFO - feeder - thread feeder-001 exit
2021-06-21 00:27:06,217 - INFO - icrawler.crawler - starting 1 parser threads...
2021-06-21 00:27:06,222 - INFO - icrawler.crawler - starting 1 downloader threads...
2021-06-21 00:27:06,508 - INFO - parser - parsing result page https://www.bing.com/images/async?q=猫&first=0
2021-06-21 00:27:12,096 - ERROR - downloader - Exception caught when downloading file https://img.zcool.cn/community/0188a25d3900e8a8012187f4a517f2.jpg, error: HTTPSConnectionPool(host='img.zcool.cn', port=443): Read timed out. (read timeout=5), remaining retry times: 2
2021-06-21 00:27:14,291 - INFO - downloader - image #1	https://img.zcool.cn/community/0188a25d3900e8a8012187f4a517f2.jpg
2021-06-21 00:27:15,280 - INFO - downloader - image #2	https://img.zcool.cn/community/01a1365c7cda34a801213f26fe09b3.j

In [8]:
# -*- coding: utf-8 -*-

from PIL import Image

import os, glob

import numpy as np

from PIL import ImageFile

# IOError: image file is truncated (0 bytes not processed)回避のため

ImageFile.LOAD_TRUNCATED_IMAGES = True



classes = ["dog", "cat"]

num_classes = len(classes)

image_size = 64

num_testdata = 25



X_train = []

X_test  = []

y_train = []

y_test  = []



for index, classlabel in enumerate(classes):

    photos_dir = "./" + classlabel

    files = glob.glob(photos_dir + "/*.jpg")

    for i, file in enumerate(files):

        image = Image.open(file)

        image = image.convert("RGB")

        image = image.resize((image_size, image_size))

        data = np.asarray(image)

        if i < num_testdata:

            X_test.append(data)

            y_test.append(index)

        else:



            # angleに代入される値

            # -20

            # -15

            # -10

            #  -5

            # 0

            # 5

            # 10

            # 15

            # 画像を5度ずつ回転            

            for angle in range(-20, 20, 5):



                img_r = image.rotate(angle)

                data = np.asarray(img_r)

                X_train.append(data)

                y_train.append(index)

                # FLIP_LEFT_RIGHTは 左右反転

                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)

                data = np.asarray(img_trains)

                X_train.append(data)

                y_train.append(index)



X_train = np.array(X_train)

X_test  = np.array(X_test)

y_train = np.array(y_train)

y_test  = np.array(y_test)



xy = (X_train, X_test, y_train, y_test)

np.save("./dog_cat.npy", xy)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [9]:
# -*- coding: utf-8 -*-

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D

from keras.layers import Activation, Dropout, Flatten, Dense

# from keras.optimizers import RMSprop

from keras.utils import np_utils

import keras

import numpy as np



classes = ["dog", "cat"]

num_classes = len(classes)

image_size = 64



"""

データを読み込む関数

"""

def load_data():

    X_train, X_test, y_train, y_test = np.load("./dog_cat.npy", allow_pickle=True)

    # 入力データの各画素値を0-1の範囲で正規化(学習コストを下げるため)

    X_train = X_train.astype("float") / 255

    X_test  = X_test.astype("float") / 255

    # to_categorical()にてラベルをone hot vector化

    y_train = np_utils.to_categorical(y_train, num_classes)

    y_test  = np_utils.to_categorical(y_test, num_classes)



    return X_train, y_train, X_test, y_test



"""

モデルを学習する関数

"""

def train(X, y, X_test, y_test):

    model = Sequential()

    # Xは(1200, 64, 64, 3)

    # X.shape[1:]とすることで、(64, 64, 3)となり、入力にすることが可能です。

    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))

    model.add(Activation('relu'))

    model.add(Conv2D(32,(3,3)))

    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Dropout(0.1))



    model.add(Conv2D(64,(3,3), padding='same'))

    model.add(Activation('relu'))

    model.add(Conv2D(64,(3,3)))

    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Dropout(0.25))



    model.add(Flatten())

    model.add(Dense(512))

    model.add(Activation('relu'))

    model.add(Dropout(0.45))

    model.add(Dense(2))

    model.add(Activation('softmax'))



    # https://keras.io/ja/optimizers/

    # 今回は、最適化アルゴリズムにRMSpropを利用

    # opt = RMSprop(lr=0.00005, decay=1e-6)

    # https://keras.io/ja/models/sequential/

    model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

    model.fit(X, y, batch_size=28, epochs=40)

    # HDF5ファイルにKerasのモデルを保存

    model.save('./cnn.h5')



    return model



"""

メイン関数

データの読み込みとモデルの学習を行います。

"""

def main():

    # データの読み込み

    X_train, y_train, X_test, y_test = load_data()



    # モデルの学習

    model = train(X_train, y_train, X_test, y_test)



main()



Epoch 1/40
52/52 [==============================] - 2s 11ms/step - loss: 0.6946 - accuracy: 0.5795
Epoch 2/40
52/52 [==============================] - 0s 10ms/step - loss: 0.6946 - accuracy: 0.6339
Epoch 3/40
52/52 [==============================] - 1s 10ms/step - loss: 0.4771 - accuracy: 0.7472
Epoch 4/40
52/52 [==============================] - 0s 9ms/step - loss: 0.3347 - accuracy: 0.8587
Epoch 5/40
52/52 [==============================] - 1s 10ms/step - loss: 0.2687 - accuracy: 0.8820
Epoch 6/40
52/52 [==============================] - 0s 9ms/step - loss: 0.1232 - accuracy: 0.9556
Epoch 7/40
52/52 [==============================] - 1s 10ms/step - loss: 0.1590 - accuracy: 0.9410
Epoch 8/40
52/52 [==============================] - 1s 10ms/step - loss: 0.0673 - accuracy: 0.9809
Epoch 9/40
52/52 [==============================] - 1s 10ms/step - loss: 0.0199 - accuracy: 0.9945
Epoch 10/40
52/52 [==============================] - 1s 10ms/step - loss: 0.0416 - accuracy: 0.9810
Epoch 11/40

In [10]:
# -*- coding: utf-8 -*-

import tensorflow as tf



model=tf.keras.models.load_model('./cnn.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open("./cnn.tflite", 'wb') as o_:

    o_.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpwtun638p/assets


2021-06-21 00:33:36,969 - INFO - tensorflow - Assets written to: /tmp/tmpwtun638p/assets


In [11]:
# -*- coding: utf-8 -*-

import tensorflow as tf

import numpy as np



def representative_data_gen():

  X_train, X_test, y_train, y_test = np.load("./dog_cat.npy", allow_pickle=True)

  X_train = X_train.astype("float32") / 255

  for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):

    yield [input_value]



model=tf.keras.models.load_model('./cnn.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.representative_dataset = representative_data_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.inference_input_type = tf.uint8

converter.inference_output_type = tf.uint8

converter.optimizations = [tf.lite.Optimize.DEFAULT]



tflite_model = converter.convert()

with open("./cnn_IntegerQuantization.tflite", 'wb') as o_:

  o_.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpura4su4n/assets


2021-06-21 00:34:44,050 - INFO - tensorflow - Assets written to: /tmp/tmpura4su4n/assets
2021-06-21 00:34:47,080 - WARNING - absl - For model inputs containing unsupported operations which cannot be quantized, the `inference_input_type` attribute will default to the original type.


In [12]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | tee /etc/apt/sources.list.d/coral-edgetpu.list
!apt-get update
!apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   117k      0 --:--:-- --:--:-- --:--:--  117k
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/re

In [13]:
!edgetpu_compiler cnn_IntegerQuantization.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 709 ms.

Input model: cnn_IntegerQuantization.tflite
Input size: 6.20MiB
Output model: cnn_IntegerQuantization_edgetpu.tflite
Output size: 6.29MiB
On-chip memory used for caching model parameters: 110.00KiB
On-chip memory remaining for caching model parameters: 7.10MiB
Off-chip memory used for streaming uncached model parameters: 6.13MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 12
Operation log: cnn_IntegerQuantization_edgetpu.log
See the operation log file for individual operation details.
